## **Qwen Code**

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q qwen-vl-utils accelerate bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.5/534.5 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 23.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info
from PIL import Image

class ColabChartAnalyzer:
    def __init__(self, model_id="Qwen/Qwen2.5-VL-7B-Instruct"):
        print(f"Loading {model_id} in 4-bit mode (Colab Safe)...")

        # 1. Define Quantization Config (This saves memory)
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )

        # 2. Load Model without Flash Attention 2 (Compatible with T4)
        self.model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map="auto"
        )

        self.processor = AutoProcessor.from_pretrained(model_id)
        print("Model loaded successfully!")

    def analyze(self, image_path, prompt_text="Analyze this chart."):
        # Load image locally or from URL
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image_path},
                    {"type": "text", "text": prompt_text},
                ],
            }
        ]

        # Prepare inputs
        text = self.processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        image_inputs, video_inputs = process_vision_info(messages)

        inputs = self.processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        )
        inputs = inputs.to("cuda")

        # Generate
        generated_ids = self.model.generate(**inputs, max_new_tokens=2024)

        # Decode
        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        output_text = self.processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )

        return output_text[0]

# --- Run the Analysis ---
if __name__ == "__main__":
    # Initialize
    analyzer = ColabChartAnalyzer()

    # Example: Analyze a sample chart from the web
    sample_chart_url = "/content/Graph1.png"

    print("\n--- Analyzing Chart ---")
    insight = analyzer.analyze(
        sample_chart_url,
        prompt_text="Describe the trend in this chart and identify the highest peak."
    )
    print(insight)

Loading Qwen/Qwen2.5-VL-7B-Instruct in 4-bit mode (Colab Safe)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Model loaded successfully!

--- Analyzing Chart ---


Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


The chart depicts the "Power Deviation over Scaled 15-Day Time Axis" from June 6, 2025, to November 18, 2025. The y-axis represents the power deviation, ranging from -100 to 100, while the x-axis represents the date.

### Trend Analysis:
1. **Initial Period (June 6 - July 14):**
   - The chart shows significant fluctuations with high peaks and troughs.
   - There is a noticeable increase around mid-June, followed by a decline towards the end of June.
   - A sharp rise occurs around July 1, peaking at approximately 90 on the y-axis.

2. **Mid-July to Mid-August:**
   - The trend stabilizes somewhat but remains volatile.
   - There is a slight dip around mid-July, followed by another rise towards the end of July.
   - Another peak is observed around August 1, reaching about 80 on the y-axis.

3. **Mid-August to Mid-September:**
   - The trend continues to fluctuate but with less extreme peaks compared to the initial period.
   - There is a noticeable drop around August 15, followed by a 

In [ ]:
# 1. Update this to match the exact name of the file you just uploaded
# Note: In Colab, uploaded files usually sit in /content/
local_image_path = "/content/Graph1.png"

# 2. Run the analysis using the EXISTING 'analyzer' (no reloading!)
try:
    print(f"Analyzing {local_image_path}...")

    insight = analyzer.analyze(
        image_path=local_image_path,
        prompt_text="Could you describe the idle periods on this graph(where the duration between datapoints is unusually high). Give their time durations as well"
    )

    print("\n--- Success! ---")
    print(insight)

except FileNotFoundError:
    print("Error: Could not find the file. Did you upload it to the Files sidebar?")
    print("Make sure the filename in the code matches the file in the sidebar exactly.")
except NameError:
    print("Error: The 'analyzer' is missing. You might have restarted the runtime.")
    print("If this happens, you sadly DO need to run the setup cell again.")

Analyzing /content/Graph1.png...


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# ---------------------------------------------------------
# RUN THIS CELL AS MANY TIMES AS YOU WANT
# (The model is already loaded in memory, so this is fast!)
# ---------------------------------------------------------

# 1. OPTIONAL: Update the image path (or keep it the same to ask more about the same chart)
current_image = "/content/Graph1.png"
# OR use the previous one:
# current_image = local_image_path

# 2. Type your NEW question here
new_prompt = "Could you describe the idle periods on this graph(where the duration between datapoints is unusually high). Give their time durations as well"

# 3. Get the answer
try:
    print(f"Asking: '{new_prompt}'...\n")

    # We use the EXISTING 'analyzer' variable.
    # Notice we do NOT say 'analyzer = ColabChartAnalyzer()' here.
    answer = analyzer.analyze(
        image_path=current_image,
        prompt_text=new_prompt
    )

    print("--- Answer ---")
    print(answer)

except NameError:
    print("⚠️ Error: The 'analyzer' variable is gone!")
    print("Did you restart the runtime? You might need to run the 'Setup' cell at the top once more.")

Asking: 'Could you describe the idle periods on this graph(where the duration between datapoints is unusually high). Give their time durations as well'...

--- Answer ---
The graph shows fluctuations in power deviation over a 15-day time axis. Idle periods, where the duration between data points is unusually high, can be identified by observing segments of the graph where there is little to no change in the power deviation values.

Here are the idle periods and their approximate durations:

1. **June 2025**:
   - The period from June 6 to June 13 appears to have an idle period. The power deviation values fluctuate but remain relatively low and stable during this period.
   - **Duration**: Approximately 7 days (from June 6 to June 13).

2. **July 2025**:
   - There is another idle period from July 14 to July 21. During this period, the power deviation values are relatively low and stable.
   - **Duration**: Approximately 8 days (from July 14 to July 21).

3. **August 2025**:
   - The pe

In [4]:
# ---------------------------------------------------------
# RUN THIS CELL AS MANY TIMES AS YOU WANT
# (The model is already loaded in memory, so this is fast!)
# ---------------------------------------------------------

# 1. OPTIONAL: Update the image path (or keep it the same to ask more about the same chart)
current_image = "/content/Graph1.png"
# OR use the previous one:
# current_image = local_image_path

# 2. Type your NEW question here
new_prompt = "Are there any significant spikes or anomalies in the power deviation data?"

# 3. Get the answer
try:
    print(f"Asking: '{new_prompt}'...\n")

    # We use the EXISTING 'analyzer' variable.
    # Notice we do NOT say 'analyzer = ColabChartAnalyzer()' here.
    answer = analyzer.analyze(
        image_path=current_image,
        prompt_text=new_prompt
    )

    print("--- Answer ---")
    print(answer)

except NameError:
    print("⚠️ Error: The 'analyzer' variable is gone!")
    print("Did you restart the runtime? You might need to run the 'Setup' cell at the top once more.")

Asking: 'Are there any significant spikes or anomalies in the power deviation data?'...

--- Answer ---
Yes, there are significant spikes and anomalies in the power deviation data. Here are some notable points:

1. **Early June**: There is a large spike around June 6th where the power deviation reaches approximately 90.
2. **Mid-June to Early July**: There is another significant spike around June 21st, where the power deviation peaks at about 80.
3. **Mid-July**: There is a noticeable drop followed by a small spike around July 7th.
4. **Mid-August**: There is a large spike around August 5th, reaching up to 100.
5. **Mid-September**: There is another significant spike around September 19th, reaching up to 90.
6. **Mid-October**: There is a sharp drop followed by a small spike around October 5th.
7. **Mid-November**: There is a large spike around November 18th, reaching up to 80.

These spikes indicate periods of high deviation from the expected power levels, which could be due to variou

In [6]:
# ---------------------------------------------------------
# RUN THIS CELL AS MANY TIMES AS YOU WANT
# (The model is already loaded in memory, so this is fast!)
# ---------------------------------------------------------

# 1. OPTIONAL: Update the image path (or keep it the same to ask more about the same chart)
current_image = "/content/Graph1.png"
# OR use the previous one:
# current_image = local_image_path

# 2. Type your NEW question here
new_prompt = "After the maintenance event on the 28th of August,2025, did power deviation reduce and by how much? Use 30-day moving average to compare"

# 3. Get the answer
try:
    print(f"Asking: '{new_prompt}'...\n")

    # We use the EXISTING 'analyzer' variable.
    # Notice we do NOT say 'analyzer = ColabChartAnalyzer()' here.
    answer = analyzer.analyze(
        image_path=current_image,
        prompt_text=new_prompt
    )

    print("--- Answer ---")
    print(answer)

except NameError:
    print("⚠️ Error: The 'analyzer' variable is gone!")
    print("Did you restart the runtime? You might need to run the 'Setup' cell at the top once more.")

Asking: 'After the maintenance event on the 28th of August,2025, did power deviation reduce and by how much? Use 30-day moving average to compare'...

--- Answer ---
To determine if the power deviation reduced after the maintenance event on the 28th of August, 2025, and by how much, we can use a 30-day moving average to smooth out the data and identify trends more clearly.

Here's a step-by-step approach:

1. **Calculate the 30-Day Moving Average**: Compute the moving average for each day starting from the first day of the dataset up to the last day. This will give us a smoothed version of the power deviation over time.

2. **Identify the Maintenance Event**: The maintenance event occurred on the 28th of August, 2025. We need to look at the moving averages around this date to see if there is a significant change.

3. **Compare the Moving Averages**: Compare the moving averages before and after the maintenance event to see if there is a reduction in power deviation.

Let's go through th

In [7]:
# Re-import this just in case (it's fast/instant)
from qwen_vl_utils import process_vision_info

def analyze_long(analyzer_instance, image_path, prompt, max_tokens=2048):
    """
    A standalone function that uses your EXISTING analyzer
    but allows for longer responses.
    """
    print(f"Generating up to {max_tokens} tokens...")

    # 1. Prepare the message
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": prompt},
            ],
        }
    ]

    # 2. Process Inputs (Using the processor from your existing analyzer)
    text = analyzer_instance.processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = analyzer_instance.processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # 3. Generate with CUSTOM max_tokens
    # We access the model inside your analyzer instance directly
    generated_ids = analyzer_instance.model.generate(**inputs, max_new_tokens=max_tokens)

    # 4. Decode
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = analyzer_instance.processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    return output_text[0]

# --- How to use it ---
# Now you can call this function and pass your 'analyzer' variable to it
long_insight = analyze_long(
    analyzer_instance=analyzer,        # Pass your existing loaded bot
    image_path=current_image,       # Your chart path
    prompt="After the maintenance event on the 28th of August,2025, did power deviation reduce and by how much? Use 30-day moving average to compare",
    max_tokens=2000                    # <-- CHANGE THIS NUMBER FREELY
)

print("\n--- Long Analysis ---")
print(long_insight)

Generating up to 2000 tokens...

--- Long Analysis ---
To determine if the power deviation reduced after the maintenance event on the 28th of August, 2025, and by how much, we can use a 30-day moving average to smooth out the data and identify trends more clearly.

Here's a step-by-step approach:

1. **Calculate the 30-Day Moving Average**: Compute the moving average for each day starting from the first day of the dataset up to the last day. This will give us a smoothed version of the data where each point represents the average of the previous 30 days.

2. **Identify the Maintenance Event**: The maintenance event occurred on the 28th of August, 2025. We need to look at the moving averages around this date.

3. **Compare the Moving Averages**: Compare the moving averages before and after the maintenance event to see if there is a reduction in power deviation.

Let's go through these steps with the provided data:

### Step 1: Calculate the 30-Day Moving Average

For simplicity, let's as